In [ ]:
print(df[df['Gender'].isnull()])
df['Gender'] = df['Gender'].fillna(df['Gender'].dropna().mode().values[0] )

df.loc[[23,126,171,188,314,334,460,467,477,507,576,588,592],'Gender']


In [ ]:
sns.catplot(
    data=df, x="Credit_History", y="LoanAmount", col="Gender",
    kind="bar", height=4, aspect=.6,
)

sns.countplot(x= df['Credit_History'] ,hue= df['Loan_Status'])
sns.countplot(x=df["Credit_History"])


In [ ]:
sns.boxplot(y=df2['LoanAmount_Log'])
plt.title("Box plot of original Dataset")

#  Calculate the first and third quartiles (Q1 and Q3) using the quantile method
q1 = abc['LoanAmount_Log'].quantile(q=0.25)
q3 = abc['LoanAmount_Log'].quantile(q=0.75)

# Calculate iqr (iqr = q3 - q1)
iqr = q3 - q1

#  Calculate the lower and upper bounds for outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Identify outliers in column 'Diameter (micrometer)'
outliers = abc[(abc['LoanAmount_Log'] < lower_bound) | (abc['LoanAmount_Log'] > upper_bound)]

# Print the outliers
print("Outliers in column 'LoanAmount_Log':")
outliers

def z_score(abc: pd.DataFrame, col_name: str) -> pd.DataFrame:
    """
    This function uses Z-score outlier detection
    method to detect outliers and retrn them in a data frame
    
    :param: Takes a data frame and the column name
    
    :return: list of the outliers you had
    in your data frame
    """
    
    std: np.float16 = np.float16(abc[col_name].std())
    mean: np.float16 = np.float16(abc[col_name].mean())
    
    upper_limit = mean + (3 * std)
    lower_limit = mean - (3 * std)
    
    outliers_free = [value for value in abc[col_name] if 
                  not((value > upper_limit) | (value < lower_limit))]
    

    filtered_data = abc.loc[abc[col_name].isin(outliers_free)]
    
    
    return filtered_data
---------------------------------


abc = z_score(abc, "ApplicantIncome")
abc = z_score(abc, "CoapplicantIncome")

fig, axes = plt.subplots(nrows=2, ncols=2)

sns.violinplot(x =abc["LoanAmount"], ax = axes[1,0], color= "#b3b3ff")
sns.violinplot(x= abc["Loan_Amount_Term"], ax = axes[1,1], color= "#b3ffd9")
sns.violinplot(x= abc["ApplicantIncome"], ax = axes[0,0], color= "#ffcccc")
sns.violinplot(x= abc["CoapplicantIncome"], ax = axes[0,1], color= "#e6b3ff")

fig.set_size_inches(10,6)
fig.suptitle("Make sure we fixed outliers", fontsize= font)
plt.show()

In [ ]:
# Grid search method
def svr_model(X, y):
    gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid={
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    best_svr = SVR(kernel='rbf', C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)

    scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error'}

    scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
    return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean()))



print(svr_model(X,y))

https://www.kaggle.com/code/kshitij192/hyperparameter-tuning-using-grid-search-cv

Best resource
